<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/twadr_30percent_mixed_pubmed_bigtweet_twadr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'tward_training_folder_0.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"tward_validation_folder_0.csv",header=None)
valid.head()


,0,1
0,1811,cant shut up for the whole day
1,903,angry
2,1576,got me to the ceiling
3,2201,anxiety
4,416,tingling


In [12]:
# testing data set

test = pd.read_csv(path/"tward_testing_overlap_30.csv",header=None)
test.head()

,0,1
0,1576,still not tired
1,2099,can't remember the last time i slept
2,1698,i feel like
3,1811,wired
4,476,still procrastinating


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_twadr.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'was',
 'diclofenac',
 'were',
 'on',
 'is',
 'rt',
 'pain',
 'by',
 'sodium',
 'drug',
 'that',
 'as',
 'i',
 'at',
 'or',
 'patients',
 'from',
 'this',
 'you',
 'disease',
 'be',
 'increased',
 'mg',
 'my',
 'group',
 'study',
 'mood',
 'are',
 'not',
 'treatment',
 'it',
 'an',
 'after',
 'me',
 'have',
 'amp',
 'p',
 'release',
 'gluten',
 ',',
 'disorder',
 'all',
 'effect',
 'free',
 'abnormal',
 'we',
 'withdrawal',
 'than',
 'no',
 'symptoms',
 'weight',
 'your',
 'new',
 'compared',
 'decreased',
 'anxiety',
 'out',
 'more',
 'using',
 'time',
 'effects',
 'groups',
 'day',
 'up',
 'two',
 'our',
 'about',
 'both',
 'blood',
 'inflammatory',
 'its',
 'drugs',
 'significantly',
 'between',
 'h',
 'like',
 'potassium',
 'acute',
 'had',
 'these',
 'significant',
 'but',
 'can',
 'use',
 'anti',
 'disorders',
 'skin',
 'which',
 '

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.731084,4.207147,0.342756,22:45


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.003068,3.834535,0.379403,22:39


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.727252,3.789739,0.382731,22:44


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.678767,3.758499,0.386688,22:41


In [25]:
learn.save_encoder('mixed_pubmed_bigtweet_twadr_fold30overlap_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('mixed_pubmed_bigtweet_twadr_fold30overlap_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.152693,6.161470,0.147826,00:22


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.147120,5.014022,0.130435,00:25


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.691966,4.721329,0.165217,01:06


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.373459,4.376959,0.191304,01:06
1,5.137121,4.187041,0.234783,01:06


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.867681,4.006854,0.260870,01:02
1,4.687819,3.907118,0.269565,01:09


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.488296,3.802690,0.252174,01:05
1,4.311838,3.726441,0.295652,01:07


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.133761,3.597434,0.278261,01:08
1,3.981192,3.616763,0.286957,01:08


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.794875,3.580955,0.295652,01:04
1,3.663341,3.514416,0.304348,01:05


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.480263,3.523776,0.330435,01:09
1,3.388734,3.475458,0.339130,01:05
2,3.187904,3.394311,0.347826,01:10
3,3.037448,3.367830,0.339130,01:07


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.871193,3.441291,0.313043,01:06
1,2.825622,3.466261,0.356522,01:09
2,2.659356,3.349174,0.365217,01:07
3,2.480978,3.372104,0.347826,01:08


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.303961,3.475636,0.339130,01:09
1,2.296996,3.487032,0.339130,01:07
2,2.139624,3.431907,0.339130,01:09
3,1.994745,3.426576,0.347826,01:05


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.875232,3.479036,0.365217,01:07
1,1.834843,3.517650,0.330435,01:08
2,1.691335,3.436597,0.347826,01:08
3,1.547372,3.441474,0.347826,01:05


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.421511,3.432644,0.339130,01:08
1,1.392425,3.481372,0.347826,01:10
2,1.292436,3.510684,0.356522,01:09
3,1.212258,3.518645,0.347826,01:04


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.092098,3.625310,0.365217,01:03
1,1.128405,3.646880,0.365217,01:10
2,1.032293,3.659115,0.347826,01:09
3,0.942030,3.715503,0.347826,01:05


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.872141,3.755917,0.330435,01:05
1,0.878272,3.952573,0.339130,01:06
2,0.845175,3.893560,0.330435,01:09
3,0.766369,3.939038,0.304348,01:03


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.730575,3.877970,0.347826,01:10
1,0.708142,4.001447,0.313043,01:03


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.658510,3.919621,0.330435,01:07
1,0.620358,4.090423,0.313043,01:05


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.588693,3.967819,0.347826,01:11


In [44]:
# save the best model

learn.save_encoder('mixed_pubmed_bigtweet_twadr_fold30overlap')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

205
205
[803, 2099, 1308, 1576, 1932, 359, 700, 803, 854, 309, 965, 135, 375, 375, 2188, 1534, 854, 711, 965, 1674, 2046, 803, 744, 1811, 1802, 171, 1702, 974, 833, 1811, 378, 1411, 536, 1576, 12, 1811, 1811, 244, 244, 1576, 803, 1822, 1652, 1950, 92, 1935, 1307, 621, 1323, 854, 1811, 574, 700, 1723, 135, 1323, 1583, 194, 1932, 1308, 92, 700, 1932, 700, 909, 744, 1968, 2097, 2027, 2188, 2099, 1811, 309, 1403, 1570, 1698, 79, 2099, 978, 574, 724, 1038, 92, 476, 909, 1106, 135, 1698, 1307, 359, 230, 451, 1932, 1106, 621, 1106, 1570, 798, 723, 700, 965, 1106, 1811, 1034, 1034, 1811, 1478, 2088, 476, 635, 1411, 1576, 854, 1639, 1411, 2099, 2214, 1652, 574, 2165, 1811, 1178, 2088, 1136, 268, 1313, 2099, 1811, 1307, 814, 170, 135, 574, 309, 1811, 1534, 1576, 1811, 2018, 1534, 1308, 621, 1773, 170, 1811, 909, 1034, 1811, 2184, 1326, 983, 1633, 840, 170, 441, 2046, 2099, 1932, 700, 1034, 1392, 170, 135, 1308, 1376, 2046, 135, 79, 1962, 170, 2099, 79, 249, 89, 441, 909, 847, 1811, 931, 1308, 57

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

71
0.3463414634146341
